In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\HP\\Cancer Detection'

In [2]:
# set MLFLOW_TRACKING_URI = https://dagshub.com/aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps.mlflow

# set MLFLOW_TRACKING_USERNAME=aderojuabdulsalam15

# set MLFLOW_TRACKING_PASSWORD = dfa6235dcb1f78695261c06240df2bcef0ba5d58

## Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path
from urllib.parse import urlparse
import mlflow.keras


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## Configuration Manager

In [4]:
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json
from src.cnnClassifier.constants import *
import mlflow
import tensorflow as tf

#### Initiating Dagshub (Server for MLFlow)

In [5]:
import dagshub
dagshub.init(repo_owner='aderojuabdulsalam15', repo_name='End-to-end-ML-Project-with-MLOps', mlflow=True)

Initialized MLflow to track repo "aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps"

[2024-07-18 15:43:26,307: INFO: helpers: Initialized MLflow to track repo "aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps"]


Repository aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps initialized!

[2024-07-18 15:43:26,318: INFO: helpers: Repository aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps initialized!]


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data =  "artifacts/data_ingestion/Chest-CT-Scan-data",
            all_params =  self.params,
            mlflow_uri = "https://dagshub.com/aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps.mlflow",
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config
    

    

## Component

In [7]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

## Pipeline

In [8]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e
    

[2024-07-18 15:43:27,564: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-18 15:43:27,575: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-18 15:43:27,577: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 24s 3s/step - loss: 35.4628 - accuracy: 0.5686
[2024-07-18 15:43:52,846: INFO: common: json file saved at: scores.json]
[2024-07-18 15:44:14,794: WARNING: connectionpool: Retrying (Retry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018F9D06BC40>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond')': /aderojuabdulsalam15/End-to-end-ML-Project-with-MLOps.mlflow/api/2.0/mlflow/r

2024/07/18 15:44:15 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpf5isy5s5\model\data\model\assets
[2024-07-18 15:44:18,436: INFO: builder_impl: Assets written to: C:\Users\HP\AppData\Local\Temp\tmpf5isy5s5\model\data\model\assets]


c:\Users\HP\anaconda3\envs\myenv\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/07/18 15:45:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
